In [2]:
import wandb                                              # The library which will help me keep track hyperparameter tuning results
from wandb.integration.keras import WandbMetricsLogger    # WandbMetricsLogger -> This function will take the model results ( loss - evaluation accuracy - training accuracy , ..) and send it to the w&b website to store and show these current hyperparameters results

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

In [5]:
# sweep hyperparameters process  :  Try the all combinations of these hyperparameters values

sweep_config = {

    'method' : 'grid',                                 #  GridSearch ->  to try the all combinations of these hyperparameters values
    'metric' : {
        'name': 'val_accuracy',
        'goal': 'maximize'
                },
    'parameters' : {                                   # The hyperparamter values combniations which we will try  =  2 * 2 * 2 * 2 * 2 = 32 model
        'batch_size': {'values': [8,16]},
        'learning_rate': {'values': [0.001,0.0001]},
        'hidden_nodes': {'values': [128,64]},
        'img_size': {'values': [16, 224]},
        'epochs': {'values': [5,10]}
    }
}

sweep_id = wandb.sweep( sweep_config , project="flowers classification" )        # This sweep ID =  'ittyccyp'

Create sweep with ID: 6w8mndyj
Sweep URL: https://wandb.ai/https-tanta-edu-eg-/flowers%20classification/sweeps/6w8mndyj


    ✅ sweep hyperparameters process


        Means try the all combinations of these hyperparameters values so we can choose the best hyperparameters values to get the
    

        best model which will get the best evaluation accuracy




        🔴    'method' : 'grid'   ▶


                GridSearch ->  to try the all combinations of these hyperparameters values   ->  2 * 2 * 2 * 2 * 2 = 32 model




        🔴    'metric' : {
                            'name': 'val_accuracy',

                            'goal': 'maximize'
                          }



              ✅ here we are telling W&B website to rank the results of these models according to the validation accuracy and


               our goal is to rank according to the maximum validation accuracy



              ✅ may be we can choose  'name' : 'loss' , 'goal' : 'minimize' , here we are telling W&B website to store the


              results of these models according to the loss ( averge error between actuals , predicted probabilites ) and


              our goal id to rank from the models with the minimim loss ( best ) to the models with the maximum loss ( worst )






        🔴  sweep_id = wandb.sweep( sweep_config , project = "flowers classification" )



              ✅ here we will send these configurations to the W&B servers to tell it that these configurations will be applied


              for the flowers classifications project ✅ , if this project is in the W&B servers it will connect it directlt


              to these configurations , if this project name isn't exist in the W&B servers it will create a new project called


              flowers configuration and connect it with these configurations which in the sweep_donfig variable and we will find


              this project in the projects at W&B website  




              ✅  After that the server will respond with ID which we will store in the sweep_id variable and we will use this ID


              as an identifier to connect between colab and w&b server in this flower classification project trials , so that


              we can exchange the values between colab and w&b servers by getting the current run parameters values from


              w&b servers to start another training with these new hyperparameters values , then send this current


              run results from colab to w&b servers so it can store and visualize it .




              ❗  Note :  Each time we run this sweep_id code it will create another ID and any calculations happened and stored


              from the previous ID will vanish ( so we have to store the ID of this stage if we want to keep continue from we  


              stopped even i will close the labtop to sleep or any other thing , i have to store the id of this stage to put  


              it here wandb.agent( "YOUR_OLD_SWEEP_ID" , function = train ) ✅ , to keep continue from the (last run - iteration)


              i stopped at before closing the laptop or before any crash , because these results is stored in a cloud and


              connected with this ID ✅ , so we have to use it again if we don't want to start from scratch again at the  
              
              
              hyperparameter tuning , but we will start from the last trial .





              ✅ for short swap_id is a bridge between our models training results and the W&B website to store and show it for us

In [6]:
def train():                     # Model Training function
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        keras.layers.Dense(config.hidden_nodes, activation="relu"),
        keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5)]
      )

    
    
    ✅  with wandb.init() as run:


          with  :  we use it to start each trial ( hyperparameters combination ) alone ❗ then when it end , we will send it's


          model results ( loss - acccuracy , etc ) to the W&B server then return back and start the next hyperparameters values
          
          
          then when it end it sends this model results to the W&B server and return again to the code an so on till all


          combinations end  




          wand.init()  : we are telling W&B server to start a new session with the new hyperparameters values



          as run  :  It's a variable which store the current model ( combination ) results




    ✅  config = wandb.config


          At each new model building , this line will go to the W&B server to get the hyperparameters values of this new model


          ( the values of the hyperparameters for this trial ) which we sent to the W&B server in the sweep_config variable ✅


          So each trial the config variable will store the new hyperparameters combination values for this new model .

In [8]:
wandb.agent( 'ittyccyp' , function = train )                 # sweep_id = 'ittyccyp'

    195/Unknown 77s 391ms/step - accuracy: 0.2965 - loss: 1.5717

wandb: Agent Starting Run: 5dusy4kp with config:
wandb: 	batch_size: 16
wandb: 	epochs: 10
wandb: 	hidden_nodes: 64
wandb: 	img_size: 16
wandb: 	learning_rate: 0.001
Traceback (most recent call last):
  File "/tmp/ipython-input-1731012624.py", line 57, in train
    model.fit(
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
  File "/usr/local/lib/python3.12/dist-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/usr/local/lib/python3.12/dist-packages/wandb/integration/keras/callbacks/metrics_logger.py", line 129, in on_train_batch_end
    self.on_batch_end(batch, logs if logs else {})
  File "/usr/local/lib/python3.12/dist-packages/wandb/integration/keras/callbacks/metrics_logger.py", line 121, in on_batch_end
    wandb.log(logs)
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 400, in wrapper
    return func(self, *args, **kwargs)
         

    wandb.log(logs)
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 400, in wrapper


return func(self, *args, **kwargs)


           ^^Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 336, in check_internal_messages
    self._loop_check_status(
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/wandb_run.py", line 237, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/interface/interface.py", line 1007, in deliver_internal_messages
^    return self._deliver_internal_messages(internal_message)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/wandb/sdk/interface/interface_shared.py", line 499, in _deliver_internal_messages
    return self._deliver(record)
           ^^

...

Epoch 1/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 148s 713ms/step - accuracy: 0.2951 - loss: 13.5126 - val_accuracy: 0.3270 - val_loss: 4.4738
Epoch 2/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 145s 697ms/step - accuracy: 0.3393 - loss: 2.4225 - val_accuracy: 0.2568 - val_loss: 1.5934
Epoch 3/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 147s 708ms/step - accuracy: 0.3396 - loss: 1.5133 - val_accuracy: 0.3000 - val_loss: 1.5477
Epoch 4/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 147s 707ms/step - accuracy: 0.3402 - loss: 1.4768 - val_accuracy: 0.2892 - val_loss: 1.5165
Epoch 5/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 141s 681ms/step - accuracy: 0.3517 - loss: 1.4497 - val_accuracy: 0.3216 - val_loss: 1.4823
Epoch 6/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 203s 983ms/step - accuracy: 0.3268 - loss: 1.4459 - val_accuracy: 0.3027 - val_loss: 1.4822
Epoch 7/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 144s 693ms/step - accuracy: 0.3263 - loss: 1.4415 - val_accuracy: 0.3081 - val_loss: 1.4748
Epoch 8/10
207/207 ━━━━━━━━━━━━━━━━━━━━ 141s 679ms/step - accuracy: 0.3355 

batch/accuracy,▂▃▄▄▂▂▃▃▄▅▅▄▅▄▄▃▄▃▅▅▅▃▃▃▃▃▃█▁▄▄▄▄▃▃▃▃▃▂▃
batch/batch_step,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▅▅▄▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▂▁█▅▇▅▅▄▁▃
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▁▁▁▁▁
epoch/val_accuracy,█▁▅▄▇▅▆█▃█
epoch/val_loss,█▁▁▁▁▁▁▁▁▁
batch/accuracy,0.31948


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7f342c2762a0>> (for post_run_cell):


AlreadyJoinedError: 



```

✅  wandb.agent role   ▶


It's the bridge between the notebook and the W&B server  ▶


*   First it connects to the server using the sweep_Id ✅ which we got from the server


*   Then it ask the server about the current trial and it's hyperparameters values


*   After Getting these values it pass it the train function and starts to execute it's code




The agent function ✅ is entering a loop to execute the all combinations and it follows these steps  ▶


      ▶  At the first loop it Get the current parameters values from the server


      ▶  run the the train function


      ▶  when finish this trial it sends the results to the server , then end this trial
      
      
      ▶ Then start run 2 and go to the W&B server to get the new parameters values and run the train function , send the  
      
      
      results to the server  , fininsh this trial , then start the 3 run and so on till fininsh the all
      
      
      combinations in the sweep_config which stored in the server ( 32 iteration  -  32 runs ) ✅



```

